# Seam Carving

In [1]:
#!/usr/bin/env python

import sys
import numpy as np
import tkinter as tk
import tkinter.filedialog
from tkinter import messagebox
from imageio import imread, imwrite
from scipy.ndimage.filters import convolve
from PIL import ImageTk,Image
import pickle

def calc_energy(img):
    filter_du = np.array([
        [1.0, 2.0, 1.0],
        [0.0, 0.0, 0.0],
        [-1.0, -2.0, -1.0],
    ])
    # This converts it from a 2D filter to a 3D filter, replicating the same
    # filter for each channel: R, G, B
    filter_du = np.stack([filter_du] * 3, axis=2)

    filter_dv = np.array([
        [1.0, 0.0, -1.0],
        [2.0, 0.0, -2.0],
        [1.0, 0.0, -1.0],
    ])
    # This converts it from a 2D filter to a 3D filter, replicating the same
    # filter for each channel: R, G, B
    filter_dv = np.stack([filter_dv] * 3, axis=2)

    img = img.astype('float32')
    convolved = np.absolute(convolve(img, filter_du)) + np.absolute(convolve(img, filter_dv))

    # We sum the energies in the red, green, and blue channels
    energy_map = convolved.sum(axis=2)
    return energy_map

def crop_c(img, new_c):
    r, c, _ = img.shape

    for i in range(c - new_c):
        img = carve_column(img)

    return img

def crop_r(img, new_r):
    img = np.rot90(img, 1, (0, 1))
    img = crop_c(img, new_r)
    img = np.rot90(img, 3, (0, 1))
    return img

def carve_column(img):
    r, c, _ = img.shape

    M, backtrack = minimum_seam(img)
    mask = np.ones((r, c), dtype=np.bool)

    j = np.argmin(M[-1])
    for i in reversed(range(r)):
        mask[i, j] = False
        j = backtrack[i, j]

    mask = np.stack([mask] * 3, axis=2)
    img = img[mask].reshape((r, c - 1, 3))

    return img

def minimum_seam(img):
    r, c, _ = img.shape
    energy_map = calc_energy(img)

    M = energy_map.copy()
    backtrack = np.zeros_like(M, dtype=np.int)

    for i in range(1, r):
        for j in range(0, c):
            # Handle the left edge of the image, to ensure we don't index a -1
            if j == 0:
                idx = np.argmin(M[i-1, j:j + 2])
                backtrack[i, j] = idx + j
                min_energy = M[i-1, idx + j]
            else:
                idx = np.argmin(M[i - 1, j - 1:j + 2])
                backtrack[i, j] = idx + j - 1
                min_energy = M[i - 1, idx + j - 1]

            M[i, j] += min_energy

    return M, backtrack


In [2]:
def shorten():
    DB = open('data.dat','rb')
    data = pickle.load(DB)
    DB.close()
    img = imread(data['filepath'])
    new_dim = entry1_1.get()
    r, c, _ = img.shape
    
    #regex for newdim [0-9]*x[0-9]*
    if new_dim.count('x')==1 and len(new_dim.split('x'))==2 :
        l = new_dim.split('x')
        for i in l :
            if not i.isnumeric() :
                messagebox.showinfo("WARNING !!", "Improper Format of Dimensions. Numbers NOT PROVIDED!")
                return
        if not ( 0 <= int(l[0]) <= c and 0 <= int(l[1]) <= r  ) :
            messagebox.showinfo("WARNING !!", "Requested width or height is not in proper limits. Please try again !!")
            return
    else :
        messagebox.showinfo("WARNING !!", "Improper Format of Dimensions. Format is <Width>+x+<Height>    |    For example: 123x456.")
        return
    
    #out file name
    output = tkinter.filedialog.asksaveasfile(filetypes = [('PNG','*.png')], defaultextension = [('PNG','*.png')])
    
    # ALL ALERTS PASSED
    
    # SHOW PREP
    label1_3.pack_forget()
    label1_4.pack_forget()
    label1_5.pack_forget()
    entry1_1.pack_forget()
    button1_1.pack_forget()
    imgcanvas.pack_forget()
    desc_1_2.pack_forget()
    button1_2.pack_forget()
    desc_1_1.configure(text="Shortening your image...")
    print()
    # FINAL WORK
    l = new_dim.split('x')
    img = crop_r(img, int(l[1]))
    img = crop_c(img, int(l[0]))
    imwrite(output.name, img)
    
    desc_1_1.configure(text="Image Shortened Successfully !!")
    image = Image.open(output.name)
    [imageSizeWidth, imageSizeHeight] = [ int(i) for i in l ]
    heightlimit = 200
    dh = imageSizeHeight - heightlimit
    if (dh<0):
        n = 0
    else :
        n = dh/imageSizeHeight
    n = 1-n
    newImageSizeHeight = int(imageSizeHeight*n) 
    newImageSizeWidth = int(imageSizeWidth*n)
    image = image.resize((newImageSizeWidth, newImageSizeHeight), Image.ANTIALIAS)
    img = ImageTk.PhotoImage(image)
    imgcanvas.config(image=img,height=200,width=40)
    imgcanvas.image = img
    imgcanvas.pack(fill="both", expand="yes")
    button1_2.configure(text="Exit",command=root.destroy)
    button1_2.pack()
    messagebox.showinfo("SUCCESS !!", "SUCCESS")


In [3]:
def fetchfile() :
    filepath = tk.filedialog.askopenfilename(parent=root,filetypes = [("JPEG", "*.jpg *.jpeg *.JPG *.JPEG")])
    filename = filepath.split("/")[-1]
    desc_1_2.configure(text=str("File Choosen: "+filename))
    DB = open('data.dat','wb')
    pickle.dump({'filepath':filepath,'filename':filename},DB)
    DB.close()
    if len(filename)!=0 :
        image = Image.open(filepath)
        [imageSizeWidth, imageSizeHeight] = image.size
        heightlimit = 200
        dh = imageSizeHeight - heightlimit
        if (dh<0):
            n = 0
        else :
            n = dh/imageSizeHeight
        n = 1-n
        newImageSizeHeight = int(imageSizeHeight*n) 
        newImageSizeWidth = int(imageSizeWidth*n)
        image = image.resize((newImageSizeWidth, newImageSizeHeight), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(image)
        imgcanvas.config(image=img,height=200,width=40)
        imgcanvas.image = img
        label1_3.configure(text = 'Current Size: '+str(imageSizeWidth)+'x'+str(imageSizeHeight))
        desc_1_2.pack(side=tk.TOP)
        label1_3.pack()
        label1_4.pack()
        label1_5.pack()
        entry1_1.pack(side=tk.TOP)
        pad2.pack()
        button1_2.pack()
    else :
        pass

def center_window(root,width=300, height=200):
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    x = (screen_width/2) - (width/2)
    y = (screen_height/2) - (height/2) - 30
    root.geometry('%dx%d+%d+%d' % (width, height, x, y))

root = tk.Tk()
root.columnconfigure(0, weight=1)
root.rowconfigure(1, weight=1)
root.configure(bg='#fff')
root.title("Seam Carver by cppsecrets.com")
center_window(root,800, 700)
frame = tk.Frame(root,width=root.winfo_screenwidth()-400,height=root.winfo_screenheight()-350,bg="#fff")
frame.pack(anchor=tk.N, fill=tk.X, expand=True, side=tk.TOP )

stepname = tk.Label(frame, text="Seam Carver", font=("Helvetica", 36), fg="#4f4f4f", bg='#fff' )
stepname.pack(side=tk.TOP)
cppsec = tk.Label(frame, text="by cppsecrets.com", font=("Helvetica", 12), fg="#4f4f4f", bg='#fff' )
cppsec.pack()

pad1 = tk.Label(frame,height=2,bg='#fff')
pad1.pack()

desc_1_1 = tk.Label(frame, text="Attach a JPEG image file ONLY",font=("Helvetica", 18), bg='#fff', fg="#4f4f4f", height=2 )
desc_1_1.pack(side=tk.TOP)

button1_1 = tk.Button(frame, text="Upload IMAGE", image=None, command = fetchfile, height=2, width=12, bg='#fff')
button1_1.pack()

# Image to be added here
imgcanvas = tk.Label(frame)
imgcanvas.config(height=1,width=40,bg='#fff')
imgcanvas.pack(fill="both", expand="yes")

desc_1_2 = tk.Label(frame, text='No File Choosen', font=("Helvetica", 12), fg="#4f4f4f", height=2, bg='#fff')

label1_3 = tk.Label(frame, font=("Helvetica", 18), fg="#4f4f4f", height=2, bg='#fff')
label1_4 = tk.Label(frame, text='Required Size:', font=("Helvetica", 18), fg="#4f4f4f", bg='#fff')
label1_5 = tk.Label(frame, text='Write as <Width>+x+<Height>    |    For example: 123x456', font=("Helvetica", 14), fg="#2f2f2f", bg='#fff')
entry1_1 = tk.Entry(frame)

pad2 = tk.Label(frame,height=2,bg='#fff')

button1_2 = tk.Button(frame, text="Proceed", image=None, command = shorten, height=2, width=12, bg='#fff')
tk.mainloop()
